# TODO:

### For now focus on Consumer Business
* Add Moody's methodology
* Scale according to Moody's Scales
* Standardise the Algorithm
* Build UI

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import distance


import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

# Data Analysis

In [20]:
df = pd.read_csv("JALSH Index_dataset_2000_2024_clean.csv", index_col=0, header=[0, 1])
classfier = pd.read_excel("classification_data.xlsx", index_col=0)
metrics = pd.read_excel("metrics_full.xlsx", index_col=0)

In [21]:
inverse_relationships = [False, True, False, False, True, False]

profitability = ['ebitda_margin', 'oper_margin', 'return_on_asset']
liquidity = ['tot_debt_to_ebitda', 'tot_debt_to_tot_asset', 'tot_debt_to_tot_cap', 'tot_debt_to_tot_eqy', 'interest_coverage_ratio', 'ebitda_to_tot_int_exp']
efficiency = ['invent_to_sales', 'asset_turnover']
bloomberg_metrics = ["oper_margin", 'tot_debt_to_tot_eqy', 'interest_coverage_ratio', 'return_on_asset', 'tot_debt_to_ebitda', 'ebitda_to_tot_int_exp']

In [22]:
def filter_securities(filters, data=classfier):
    
    # Apply filters directly on the transposed
    filtered_data = data.query(
        ' and '.join(f'`{k}` == "{v}"' for k, v in filters.items())
    )
    
    return list(filtered_data.index)


def calculate_overall_credit_score(credit_scores, weights = None):
    """
    Calculate the overall credit score and rating based on individual financial ratio credit scores.

    Args:
        credit_scores (list): List of credit scores for each financial ratio.

    Returns:
        tuple: A tuple containing the overall credit score (float) and overall credit rating (str).
    """
    
    max_score = 100
    min_score = 0
    
    if weights:
        overall_credit_score = sum(score * weight for score, weight in zip(credit_scores, weights)) / sum(weights)
    
    else:
        overall_credit_score = sum(credit_scores) / len(credit_scores)

    # Rescale the overall credit score to have a maximum rating of 100
    rescaled_credit_score = (overall_credit_score - min_score) / (max_score - min_score) * 100
    
    if overall_credit_score >= 75:
        overall_credit_rating = "Good"
    elif overall_credit_score >= 50:
        overall_credit_rating = "Fair"
    else:
        overall_credit_rating = "Poor"
    
    return overall_credit_score, overall_credit_rating

def calculate_credit_score(financial_ratio, industry_thresholds, expert_thresholds, inverse_relationship=False):
    """
    Calculate the actual credit score within a universal score range based on where the financial ratio
    falls within its thresholds, alongside the credit rating, confidence level, and universal range scores.
    
    Args:
        financial_ratio (float): Financial ratio value.
        industry_thresholds (dict): Dictionary of industry thresholds for each rating.
        expert_thresholds (dict): Dictionary of expert thresholds for each rating.
        inverse_relationship (bool): Indicates if the financial ratio has an inverse relationship (lower is better).

    Returns:
        tuple: A tuple containing the actual credit score (int), credit rating (str),
               confidence level (float), and universal score ranges (dict).
    """
    
    # Adjust thresholds and financial ratio for inverse relationships
    if inverse_relationship:
        financial_ratio = 1 / financial_ratio
        industry_thresholds = {k: 1/v for k, v in industry_thresholds.items()}
        expert_thresholds = {k: 1/v for k, v in expert_thresholds.items()}
    
    credit_rating = min(['good', 'fair', 'poor'], key=lambda x: distance.euclidean(
        [financial_ratio],
        [(industry_thresholds[x] + expert_thresholds[x]) / 2]
    ))
    
    # Universal score ranges for each rating category
    score_ranges = {
        'good': (75, 100),
        'fair': (50, 74),
        'poor': (0, 49)
    }

    # Determine the min and max threshold for the financial ratio
    min_threshold = min(industry_thresholds.values())
    max_threshold = max(industry_thresholds.values())
    
    # Calculate where the financial ratio falls within its thresholds
    financial_ratio = max(min_threshold, min(financial_ratio, max_threshold))
    ratio_position = (financial_ratio - min_threshold) / (max_threshold - min_threshold)
    
    # Apply this proportion to the corresponding universal score range
    min_score, max_score = score_ranges[credit_rating]
    credit_score = min_score + ratio_position * (max_score - min_score)

    # Calculate confidence level as a percentage within the score range
    confidence_level = (credit_score - min_score) / (max_score - min_score)
    confidence_level = max(0, min(1, confidence_level))
    
    return credit_score, credit_rating, confidence_level, score_ranges

def get_industry_thresholds(df, inverse_relationships, metrics=[0.25, 0.50, 0.75]):
    industry_thresholds = []

    for metric in df.index:
        q25, q50, q75 = df.loc[metric].quantile(metrics)
        
        if inverse_relationships.get(metric):
            thresholds = {"good": q25, "fair": q50, "poor": q75}
        else:
            thresholds = {"good": q75, "fair": q50, "poor": q25}
        
        industry_thresholds.append(thresholds)

    return industry_thresholds

def get_stock_metrics(df):
    dict_df = df.to_dict()
    return {stock: list(dict_df[stock].values()) for stock in dict_df}


# Score ranges
score_ranges = {
    'good': (75, 100),
    'fair': (50, 74),
    'poor': (0, 49)
}

# Function to assign categories
def assign_category(value):
    for category, (min_score, max_score) in score_ranges.items():
        if min_score <= value <= max_score:
            return category
    return 'unknown'

In [23]:
print(classfier.industry.unique())
print(classfier.sector.unique())

['Banks' 'Commercial Services' 'Electronics' 'Miscellaneous Manufactur'
 'Diversified Finan Serv' 'Building Materials' 'Mining'
 'Investment Companies' 'Pharmaceuticals' 'Beverages' 'Agriculture'
 'REITS' 'Holding Companies-Divers' 'Distribution/Wholesale' 'Food'
 'Telecommunications' 'Computers' 'Retail' 'Insurance' 'Real Estate'
 'Coal' 'Transportation' 'Entertainment' 'Auto Parts&Equipment'
 'Iron/Steel' 'Software' 'Healthcare-Services' 'Private Equity'
 'Energy-Alternate Sources' 'Forest Products&Paper' 'Internet' 'Chemicals'
 'Engineering&Construction' 'Lodging']
['Financial' 'Consumer, Non-cyclical' 'Industrial' 'Basic Materials'
 'Diversified' 'Consumer, Cyclical' 'Communications' 'Technology' 'Energy']


In [24]:
universe = {"sector":"Communications", "industry": "Telecommunications"}


In [25]:
securities = filter_securities(universe)
df = metrics.loc[bloomberg_metrics, securities]

In [26]:
# company = {'STUDY_COMPANY': {'oper_margin': 4.000646844741706,
#                              'tot_debt_to_tot_eqy': 32.80932764816569,
#                              'interest_coverage_ratio': 6.346575419010469,
#                              'return_on_asset': 4.215239913853519,
#                              'tot_debt_to_ebitda': 1.150157826723298,
#                              'ebitda_to_tot_int_exp': 7.547418258086141}}



# company = pd.DataFrame(company)
# df = df.join(company)

In [54]:
df.T.quantile([0, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

,oper_margin,tot_debt_to_tot_eqy,interest_coverage_ratio,return_on_asset,tot_debt_to_ebitda,ebitda_to_tot_int_exp
0.0,4.000647,32.809328,3.495262,4.215240,0.793112,7.547418
0.2,9.379500,47.079634,5.206050,5.526918,1.007340,8.296881
0.3,12.068926,54.214787,6.061444,6.182757,1.114453,8.671613
0.4,15.153094,58.275525,6.786826,6.602132,1.181192,9.660045
0.5,18.434634,60.799056,7.447201,6.903275,1.227743,10.955327
0.6,21.716173,63.322586,8.107577,7.204419,1.274295,12.250608
0.7,24.134574,65.016124,8.672078,8.279047,1.317894,13.138235
0.8,24.826698,65.049678,9.044832,10.900645,1.355588,13.210552
0.9,25.518823,65.083231,9.417587,13.522243,1.393282,13.282869
1.0,26.210947,65.116785,9.790341,16.143841,1.430977,13.355185


In [53]:
df.T.describe()

,oper_margin,tot_debt_to_tot_eqy,interest_coverage_ratio,return_on_asset,tot_debt_to_ebitda,ebitda_to_tot_int_exp
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,16.770215,54.881056,7.045001,8.541408,1.169894,10.703314
std,10.288422,15.246373,2.761873,5.240340,0.276201,2.968725
min,4.000647,32.809328,3.495262,4.215240,0.793112,7.547418
25%,10.724213,50.647211,5.633747,5.854837,1.060896,8.484247
50%,18.434634,60.799056,7.447201,6.903275,1.227743,10.955327
75%,24.480636,65.032901,8.858455,9.589846,1.336741,13.174394
max,26.210947,65.116785,9.790341,16.143841,1.430977,13.355185


In [51]:
np.linspace(1, 10, 10) / 1e1

array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])

In [41]:
metrics[df.columns].T.describe()

,oper_margin,return_on_asset,tot_debt_to_tot_asset,tot_debt_to_tot_cap,tot_debt_to_tot_eqy,asset_turnover,ebitda_margin,tot_debt_to_ebitda,interest_coverage_ratio,ebitda_to_tot_int_exp,invent_to_sales
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,16.770215,8.541408,21.256238,31.099454,54.881056,1.275068,26.644443,1.169894,7.045001,10.703314,3.200521
std,10.288422,5.240340,7.835156,9.404026,15.246373,0.960629,15.215842,0.276201,2.761873,2.968725,2.293524
min,4.000647,4.215240,9.576411,17.336656,32.809328,0.614033,4.984425,0.793112,3.495262,7.547418,1.624021
25%,10.724213,5.854837,20.562971,28.989338,50.647211,0.696697,21.715344,1.060896,5.633747,8.484247,1.813227
50%,18.434634,6.903275,24.565543,34.651886,60.799056,0.900477,31.493892,1.227743,7.447201,10.955327,2.306071
75%,24.480636,9.589846,25.258810,36.762002,65.032901,1.478847,36.422991,1.336741,8.858455,13.174394,3.693364
max,26.210947,16.143841,26.317455,37.757389,65.116785,2.685283,38.605563,1.430977,9.790341,13.355185,6.565921


In [42]:
metrics.T.describe()

,oper_margin,return_on_asset,tot_debt_to_tot_asset,tot_debt_to_tot_cap,tot_debt_to_tot_eqy,asset_turnover,ebitda_margin,tot_debt_to_ebitda,interest_coverage_ratio,ebitda_to_tot_int_exp,invent_to_sales
count,124.000000,124.000000,124.000000,124.000000,124.000000,124.000000,106.000000,106.000000,104.000000,106.000000,87.000000
mean,26.935606,8.030154,19.494478,30.485027,67.865970,0.840987,27.748282,4.453357,55.999754,69.060010,11.412251
std,44.586407,10.638520,12.612834,23.011801,77.627129,0.836852,23.777903,11.252781,166.694202,186.662963,8.570658
min,-15.613465,-1.982295,0.000000,0.000000,0.000000,0.016101,-13.418433,0.027125,-2.821364,-2.158024,0.000000
25%,7.751054,3.145640,9.651873,17.298927,24.450738,0.124893,11.662741,0.914727,3.602892,6.334164,3.758491
50%,16.638887,6.477793,18.417644,29.654190,44.974997,0.617119,20.192208,1.728275,7.772743,11.020836,10.881740
75%,28.805023,10.038253,28.038820,39.477491,73.431085,1.198899,35.302671,3.634308,19.821202,28.520319,16.653378
max,453.584997,109.384806,65.288960,218.185037,540.494050,4.312699,145.513193,98.162493,1339.192812,1496.857073,45.377134


In [28]:
inverse_relationships = {
    'oper_margin': False,   
    'return_on_asset': False,   
    'tot_debt_to_tot_asset': True,  
    'tot_debt_to_tot_cap': True, 
    'tot_debt_to_tot_eqy': True, 
    'asset_turnover': False,
    'ebitda_margin': False ,
    'tot_debt_to_ebitda': True,
    'interest_coverage_ratio': False,
    'ebitda_to_tot_int_exp': False, 
    'invent_to_sales': True
}


inverse_relationships = {
    'oper_margin': False,   
    'return_on_asset': False,   
    'tot_debt_to_tot_asset': True,  
    'tot_debt_to_tot_cap': True, 
    'tot_debt_to_tot_eqy': True, 
    'asset_turnover': False,
    'ebitda_margin': False ,
    'tot_debt_to_ebitda': True,
    'interest_coverage_ratio': False,
    'ebitda_to_tot_int_exp': False, 
    'invent_to_sales': True
}

In [29]:
weights_values = [.15, .20, .20, .15, .15, .15]

company_ratios = get_stock_metrics(df)
industry_thresholds = get_industry_thresholds(df, inverse_relationships)
expert_thresholds = get_industry_thresholds(df, inverse_relationships)

In [30]:
# Iterate over each company and calculate credit scores for each financial ratio
data  = {}

print(f"\t\t\t\t{universe}")
for company, ratios in company_ratios.items():
    
    
    credit_scores = []
    print(company)
    print("----" * 20)
    for i, ratio in enumerate(ratios):
        
        metric = bloomberg_metrics[i]
        
        credit_score, credit_rating, confidence_level, range_scores = calculate_credit_score(
            ratio, industry_thresholds[i], expert_thresholds[i], inverse_relationships.get(metric)
        )

        credit_scores.append(credit_score)
        
        
        print(f"Financial Ratio: {metric}: {ratio}")
        print(f"  Credit Score: {credit_score}")
        print(f"  Credit Rating: {credit_rating}")
        print(f"  Confidence Level: {confidence_level:.2%}")
        print(f"  Range Scores:")
        for rating, scores in range_scores.items():
            print(f"      {rating.capitalize()}: {scores[0]:.2f} - {scores[1]:.2f}")
        print()


    overall_credit_score, overall_credit_rating = calculate_overall_credit_score(credit_scores, weights_values)
    
    #if company == "STUDY_COMPANY":

    print(f"Overall Credit Score: {overall_credit_score:.2f}")
    print(f"Overall Credit Rating: {overall_credit_rating}")
    print()

    data[company] = credit_scores

				{'sector': 'Communications', 'industry': 'Telecommunications'}
BLU SJ Equity
--------------------------------------------------------------------------------
Financial Ratio: oper_margin: 4.000646844741706
  Credit Score: 0.0
  Credit Rating: poor
  Confidence Level: 0.00%
  Range Scores:
      Good: 75.00 - 100.00
      Fair: 50.00 - 74.00
      Poor: 0.00 - 49.00

Financial Ratio: tot_debt_to_tot_eqy: 32.80932764816569
  Credit Score: 100.0
  Credit Rating: good
  Confidence Level: 100.00%
  Range Scores:
      Good: 75.00 - 100.00
      Fair: 50.00 - 74.00
      Poor: 0.00 - 49.00

Financial Ratio: interest_coverage_ratio: 6.346575419010469
  Credit Score: 10.831550598394617
  Credit Rating: poor
  Confidence Level: 22.11%
  Range Scores:
      Good: 75.00 - 100.00
      Fair: 50.00 - 74.00
      Poor: 0.00 - 49.00

Financial Ratio: return_on_asset: 4.215239913853519
  Credit Score: 0.0
  Credit Rating: poor
  Confidence Level: 0.00%
  Range Scores:
      Good: 75.00 - 100.00
  

In [213]:
df

,BLU SJ Equity,MTN SJ Equity,TKG SJ Equity,VOD SJ Equity
oper_margin,4.000647,23.903866,12.965402,26.210947
tot_debt_to_tot_eqy,32.809328,65.116785,56.593172,65.004940
interest_coverage_ratio,6.346575,8.547827,3.495262,9.790341
return_on_asset,4.215240,7.405181,6.401370,16.143841
tot_debt_to_ebitda,1.150158,1.305329,1.430977,0.793112
ebitda_to_tot_int_exp,7.547418,13.114130,8.796523,13.355185


In [161]:
# # Apply the function to the entire dataframe
# df_categories = pd.DataFrame(data, bloomberg_metrics).applymap(assign_category)
# df_categories

In [ ]:
def calculate_credit_score(financial_ratio, industry_thresholds, expert_thresholds, inverse_relationship=False):
    """
    Calculate the actual credit score within a universal score range based on where the financial ratio
    falls within its thresholds, alongside the credit rating, confidence level, and universal range scores.
    
    Args:
        financial_ratio (float): Financial ratio value.
        industry_thresholds (dict): Dictionary of industry thresholds for each rating.
        expert_thresholds (dict): Dictionary of expert thresholds for each rating.
        inverse_relationship (bool): Indicates if the financial ratio has an inverse relationship (lower is better).

    Returns:
        tuple: A tuple containing the actual credit score (int), credit rating (str),
               confidence level (float), and universal score ranges (dict).
    """
    
    # Adjust thresholds and financial ratio for inverse relationships
    if inverse_relationship:
        financial_ratio = 1 / financial_ratio
        industry_thresholds = {k: 1/v for k, v in industry_thresholds.items()}
        expert_thresholds = {k: 1/v for k, v in expert_thresholds.items()}
        
    credit_rating = min(['good', 'fair', 'poor'], key=lambda x: distance.euclidean(
        [financial_ratio],
        [(industry_thresholds[x] + expert_thresholds[x]) / 2]
    ))

    
    #credit_rating = min(distances, key=distances.get)
    
    # Universal score ranges for each rating category
    score_ranges = {
        'good': (75, 100),
        'fair': (50, 74),
        'poor': (0, 49)
    }

    # Determine the min and max threshold for the financial ratio
    min_threshold = min(industry_thresholds.values())
    max_threshold = max(industry_thresholds.values())
    
    # Calculate where the financial ratio falls within its thresholds
    financial_ratio = max(min_threshold, min(financial_ratio, max_threshold))
    ratio_position = (financial_ratio - min_threshold) / (max_threshold - min_threshold)
    
    # Apply this proportion to the corresponding universal score range
    credit_score_range = score_ranges[credit_rating]
    score_range_width = credit_score_range[1] - credit_score_range[0]
    credit_score = credit_score_range[0] + ratio_position * score_range_width
    
    # Calculate confidence level
    confidence_level = (credit_score - credit_score_range[0]) / score_range_width
    confidence_level = max(0, min(1, confidence_level))
    
    

credit_rating = min(['good', 'fair', 'poor'], key=lambda x: distance.euclidean(
    [financial_ratio],
    [(industry_thresholds[x] + expert_thresholds[x]) / 2]
))

min_score, max_score = credit_score_range
credit_score = min_score + ratio_position * (max_score - min_score)

# Calculate confidence level as a percentage within the score range
confidence_level = (credit_score - min_score) / (max_score - min_score) * 100
confidence_level = max(0, min(100, confidence_level))


return credit_score, credit_rating, confidence_level, score_ranges

In [6]:
def calculate_credit_rating(scale, business_profile, profitability, leverage_coverage, financial_policy):
    weights = [0.2, 0.2, 0.1, 0.4, 0.1]
    weighted_average = sum(w * p for w, p in zip(weights, [scale, business_profile, profitability, leverage_coverage, financial_policy]))
    
    rating_thresholds = [(2, 'Aaa'), (3, 'Aa'), (4, 'A'), (5, 'Baa'), (6, 'Ba'), (7, 'B'), (8, 'Caa'), (float('inf'), 'Ca')]
    return next(rating for threshold, rating in rating_thresholds if weighted_average <= threshold)

rating = calculate_credit_rating(3, 3, 4, 5, 3)
print(f"The company's credit rating is: {rating}")

The company's credit rating is: A


In [7]:
def calculate_credit_rating(scale, business_profile, profitability, leverage_coverage, financial_policy):
    weights = [0.2, 0.2, 0.1, 0.4, 0.1]
    weighted_average = sum(w * p for w, p in zip(weights, [scale, business_profile, profitability, leverage_coverage, financial_policy]))
    
    rating_thresholds = [(2, 'Aaa'), (3, 'Aa'), (4, 'A'), (5, 'Baa'), (6, 'Ba'), (7, 'B'), (8, 'Caa'), (float('inf'), 'Ca')]
    rating = next(rating for threshold, rating in rating_thresholds if weighted_average <= threshold)
    
    return rating, weighted_average

rating, score = calculate_credit_rating(3, 3, 4, 5, 3)
print(f"The company's credit rating is: {rating}")
print(f"The company's credit score is: {score:.2f}")

The company's credit rating is: A
The company's credit score is: 3.90


In [ ]:
def scorecard(revenue, demand_characteristics, competitive_profile, ebita_margin, debt_ebitda, ebita_interest_expense, rcf_net_debt, financial_policy):
    """
    Calculates the scorecard-indicated outcome based on the Business and Consumer Service scorecard.
    
    Args:
        revenue (float): Total reported revenue in billions of US dollars.
        demand_characteristics (str): Description of demand characteristics (e.g., 'Reliable and steady demand').
        competitive_profile (str): Description of competitive profile (e.g., 'Multiple business segments').
        ebita_margin (float): EBITA margin as a percentage.
        debt_ebitda (float): Debt/EBITDA ratio.
        ebita_interest_expense (float): EBITA/Interest Expense ratio.
        rcf_net_debt (float): RCF/Net Debt ratio.
        financial_policy (str): Description of financial policy (e.g., 'Expected to have extremely conservative financial policies').
    
    Returns:
        str: The scorecard-indicated outcome (e.g., 'A2').
    """
    
    # Map input factors to numeric scores
    scale_score = {60.0: 'Aaa', 30.0: 'Aa', 10.0: 'A', 5.0: 'Baa', 1.5: 'Ba', 0.5: 'B', 0.2: 'Caa', 0.0: 'Ca'}
    for threshold, score in scale_score.items():
        if revenue >= threshold:
            scale_score = score
            break
    
    profitability_score = {'≥ 50%': 'Aaa', '35% - 50%': 'Aa', '25% - 35%': 'A', '20% - 25%': 'Baa', '15% - 20%': 'Ba', '10% - 15%': 'B', '5% - 10%': 'Caa', '< 5%': 'Ca'}
    for range_str, score in profitability_score.items():
        if range_str.startswith('≥'):
            if ebita_margin >= float(range_str[2:])/100:
                profitability_score = score
                break
        else:
            range_limits = [float(x)/100 for x in range_str.replace('%', '').split(' - ')]
            if range_limits[0] <= ebita_margin < range_limits[1]:
                profitability_score = score
                break
    
    leverage_coverage_scores = [
        {'< 0.5x': 'Aaa', '0.5x - 1x': 'Aa', '1x - 2x': 'A', '2x - 3x': 'Baa', '3x - 4.5x': 'Ba', '4.5x - 6.5x': 'B', '6.5x - 9x': 'Caa', '≥ 9x': 'Ca'},
        {'≥ 25x': 'Aaa', '15x - 25x': 'Aa', '10x - 15x': 'A', '6x - 10x': 'Baa', '3x - 6x': 'Ba', '1x - 3x': 'B', '0x - 1x': 'Caa', '< 0x': 'Ca'},
        {'≥ 80%': 'Aaa', '60% - 80%': 'Aa', '40% - 60%': 'A', '25% - 40%': 'Baa', '15% - 25%': 'Ba', '7.5% - 15%': 'B', '2.5% - 7.5%': 'Caa', '< 2.5%': 'Ca'}
    ]
    leverage_coverage_scores = [
        leverage_coverage_scores[0].get(str(debt_ebitda), 'Ca'),
        leverage_coverage_scores[1].get(str(ebita_interest_expense), 'Ca'),
        leverage_coverage_scores[2].get(str(rcf_net_debt), 'Ca')
    ]
    
    financial_policy_score = {'Expected to have extremely conservative financial policies': 'Aaa',
                              'Expected to have very stable and conservative financial policies': 'Aa',
                              'Expected to have predictable financial policies that preserve creditor interests': 'A',
                              'Expected to have financial policies that balance the interest of creditors and shareholders': 'Baa',
                              'Expected to have financial policies that tend to favor shareholders over creditors': 'Ba',
                              'Expected to have financial policies that favor shareholders over creditors': 'B',
                              'Expected to have financial policies that create elevated risk of debt restructuring': 'Caa'}
    financial_policy_score = financial_policy_score.get(financial_policy, 'Ca')
    
    # Calculate aggregate numeric score
    numeric_scores = [scale_score, profitability_score] + leverage_coverage_scores + [financial_policy_score]
    alpha_to_numeric = {'Aaa': 1, 'Aa': 2, 'A': 4, 'Baa': 7, 'Ba': 10.5, 'B': 13.5, 'Caa': 16.5, 'Ca': 19.5}
    numeric_score = sum(alpha_to_numeric[score] for score in numeric_scores)
    
    # Map numeric score to scorecard-indicated outcome
    scorecard_ranges = {1.5: 'Aaa', 2.5: 'Aa1', 3.5: 'Aa2', 4.5: 'Aa3', 5.5: 'A1', 6.5: 'A2', 7.5: 'A3',
                        8.5: 'Baa1', 9.5: 'Baa2', 10.5: 'Baa3', 11.5: 'Ba1', 12.5: 'Ba2', 13.5: 'Ba3',
                        14.5: 'B1', 15.5: 'B2', 16.5: 'B3', 17.5: 'Caa1', 18.5: 'Caa2', 19.5: 'Caa3'}
    for threshold, outcome in scorecard_ranges.items():
        if numeric_score < threshold:
            return outcome
    return 'Ca'

In [13]:
def scorecard(revenue, demand_characteristics, competitive_profile, ebita_margin, debt_ebitda, ebita_interest_expense, rcf_net_debt, financial_policy):
    """
    Calculates the scorecard-indicated outcome and numeric score based on the Business and Consumer Service scorecard.
    
    Args:
        revenue (float): Total reported revenue in billions of US dollars.
        demand_characteristics (str): Description of demand characteristics (e.g., 'Reliable and steady demand').
        competitive_profile (str): Description of competitive profile (e.g., 'Multiple business segments').
        ebita_margin (float): EBITA margin as a percentage.
        debt_ebitda (float): Debt/EBITDA ratio.
        ebita_interest_expense (float): EBITA/Interest Expense ratio.
        rcf_net_debt (float): RCF/Net Debt ratio.
        financial_policy (str): Description of financial policy (e.g., 'Expected to have extremely conservative financial policies').
    
    Returns:
        tuple: A tuple containing the scorecard-indicated outcome (str) and the numeric score (float).
    """
    
    # Map input factors to numeric scores
    scale_score = min((60.0, 'Aaa'), (30.0, 'Aa'), (10.0, 'A'), (5.0, 'Baa'), (1.5, 'Ba'), (0.5, 'B'), (0.2, 'Caa'), (0.0, 'Ca'), key=lambda x: x[0] if revenue >= x[0] else float('inf'))[-1]
    
    profitability_ranges = [('≥ 50%', 'Aaa'), ('35% - 50%', 'Aa'), ('25% - 35%', 'A'), ('20% - 25%', 'Baa'), ('15% - 20%', 'Ba'), ('10% - 15%', 'B'), ('5% - 10%', 'Caa'), ('< 5%', 'Ca')]
    profitability_score = next((score for range_str, score in profitability_ranges if (range_str.startswith('≥') and ebita_margin >= float(range_str[2:])/100) or (not range_str.startswith('≥') and float(range_str.split(' - ')[0])/100 <= ebita_margin < float(range_str.split(' - ')[1])/100)), 'Ca')
    
    leverage_coverage_ranges = [
        {str(x): score for x, score in zip([0.5, 1.0, 2.0, 3.0, 4.5, 6.5, 9.0], ['Aaa', 'Aa', 'A', 'Baa', 'Ba', 'B', 'Caa', 'Ca'])},
        {str(x): score for x, score in zip([25.0, 15.0, 10.0, 6.0, 3.0, 1.0, 0.0], ['Aaa', 'Aa', 'A', 'Baa', 'Ba', 'B', 'Caa', 'Ca'])},
        {str(x): score for x, score in zip([80.0, 60.0, 40.0, 25.0, 15.0, 7.5, 2.5, 0.0], ['Aaa', 'Aa', 'A', 'Baa', 'Ba', 'B', 'Caa', 'Ca'])}
    ]
    leverage_coverage_scores = [
        next((score for range_str, score in factor_ranges.items() if (range_str.startswith('≥') and factor_value >= float(range_str[2:])) or (not range_str.startswith('≥') and float(range_str.split(' - ')[0]) <= factor_value < float(range_str.split(' - ')[1]))), 'Ca')
        for factor_value, factor_ranges in [(debt_ebitda, leverage_coverage_ranges[0]), (ebita_interest_expense, leverage_coverage_ranges[1]), (rcf_net_debt, leverage_coverage_ranges[2])]
    ]
    
    financial_policy_mapping = {
        'Expected to have extremely conservative financial policies': 'Aaa',
        'Expected to have very stable and conservative financial policies': 'Aa',
        'Expected to have predictable financial policies that preserve creditor interests': 'A',
        'Expected to have financial policies that balance the interest of creditors and shareholders': 'Baa',
        'Expected to have financial policies that tend to favor shareholders over creditors': 'Ba',
        'Expected to have financial policies that favor shareholders over creditors': 'B',
        'Expected to have financial policies that create elevated risk of debt restructuring': 'Caa'
    }
    financial_policy_score = financial_policy_mapping.get(financial_policy, 'Ca')
    
    # Calculate aggregate numeric score
    alpha_to_numeric = {'Aaa': 1, 'Aa': 2, 'A': 4, 'Baa': 7, 'Ba': 10.5, 'B': 13.5, 'Caa': 16.5, 'Ca': 19.5}
    numeric_scores = [alpha_to_numeric[score] for score in [scale_score, profitability_score] + leverage_coverage_scores + [financial_policy_score]]
    numeric_score = sum(numeric_scores)
    
    # Map numeric score to scorecard-indicated outcome
    scorecard_ranges = {1.5: 'Aaa', 2.5: 'Aa1', 3.5: 'Aa2', 4.5: 'Aa3', 5.5: 'A1', 6.5: 'A2', 7.5: 'A3', 8.5: 'Baa1', 9.5: 'Baa2', 10.5: 'Baa3', 11.5: 'Ba1', 12.5: 'Ba2', 13.5: 'Ba3', 14.5: 'B1', 15.5: 'B2', 16.5: 'B3', 17.5: 'Caa1', 18.5: 'Caa2', 19.5: 'Caa3'}
    scorecard_indicated_outcome = next((outcome for threshold, outcome in scorecard_ranges.items() if numeric_score < threshold), 'Ca')
    
    return scorecard_indicated_outcome, numeric_score

In [14]:
def scorecard(revenue, demand_characteristics, competitive_profile, ebita_margin, debt_ebitda, ebita_interest_expense, rcf_net_debt, financial_policy):
    """
    Calculates the scorecard-indicated outcome and numeric score based on the Business and Consumer Service scorecard.
    
    Args:
        revenue (float): Total reported revenue in billions of US dollars.
        demand_characteristics (str): Description of demand characteristics (e.g., 'Reliable and steady demand').
        competitive_profile (str): Description of competitive profile (e.g., 'Multiple business segments').
        ebita_margin (float): EBITA margin as a percentage.
        debt_ebitda (float): Debt/EBITDA ratio.
        ebita_interest_expense (float): EBITA/Interest Expense ratio.
        rcf_net_debt (float): RCF/Net Debt ratio.
        financial_policy (str): Description of financial policy (e.g., 'Expected to have extremely conservative financial policies').
    
    Returns:
        tuple: A tuple containing the scorecard-indicated outcome (str) and the numeric score (float).
    """
    
    def get_score(value, ranges):
        for range_str, score in ranges:
            if range_str.startswith('≥') and value >= float(range_str[2:]):
                return score
            elif not range_str.startswith('≥'):
                range_min, range_max = map(float, range_str.split(' - '))
                if range_min <= value < range_max:
                    return score
        return 'Ca'
    
    scale_ranges = [(60.0, 'Aaa'), (30.0, 'Aa'), (10.0, 'A'), (5.0, 'Baa'), (1.5, 'Ba'), (0.5, 'B'), (0.2, 'Caa'), (0.0, 'Ca')]
    scale_score = get_score(revenue, scale_ranges)
    
    profitability_ranges = [('≥ 50%', 'Aaa'), ('35% - 50%', 'Aa'), ('25% - 35%', 'A'), ('20% - 25%', 'Baa'), ('15% - 20%', 'Ba'), ('10% - 15%', 'B'), ('5% - 10%', 'Caa'), ('< 5%', 'Ca')]
    profitability_score = get_score(ebita_margin, profitability_ranges)
    
    leverage_coverage_ranges = [
        [(str(x), score) for x, score in zip([0.5, 1.0, 2.0, 3.0, 4.5, 6.5, 9.0], ['Aaa', 'Aa', 'A', 'Baa', 'Ba', 'B', 'Caa', 'Ca'])],
        [(str(x), score) for x, score in zip([25.0, 15.0, 10.0, 6.0, 3.0, 1.0, 0.0], ['Aaa', 'Aa', 'A', 'Baa', 'Ba', 'B', 'Caa', 'Ca'])],
        [(str(x), score) for x, score in zip([80.0, 60.0, 40.0, 25.0, 15.0, 7.5, 2.5, 0.0], ['Aaa', 'Aa', 'A', 'Baa', 'Ba', 'B', 'Caa', 'Ca'])]
    ]
    leverage_coverage_scores = [
        get_score(factor_value, factor_ranges)
        for factor_value, factor_ranges in [(debt_ebitda, leverage_coverage_ranges[0]), (ebita_interest_expense, leverage_coverage_ranges[1]), (rcf_net_debt, leverage_coverage_ranges[2])]
    ]
    
    financial_policy_mapping = {
        'Expected to have extremely conservative financial policies': 'Aaa',
        'Expected to have very stable and conservative financial policies': 'Aa',
        'Expected to have predictable financial policies that preserve creditor interests': 'A',
        'Expected to have financial policies that balance the interest of creditors and shareholders': 'Baa',
        'Expected to have financial policies that tend to favor shareholders over creditors': 'Ba',
        'Expected to have financial policies that favor shareholders over creditors': 'B',
        'Expected to have financial policies that create elevated risk of debt restructuring': 'Caa'
    }
    financial_policy_score = financial_policy_mapping.get(financial_policy, 'Ca')
    
    alpha_to_numeric = {'Aaa': 1, 'Aa': 2, 'A': 4, 'Baa': 7, 'Ba': 10.5, 'B': 13.5, 'Caa': 16.5, 'Ca': 19.5}
    numeric_scores = [alpha_to_numeric[score] for score in [scale_score, profitability_score] + leverage_coverage_scores + [financial_policy_score]]
    numeric_score = sum(numeric_scores)
    
    scorecard_ranges = [(1.5, 'Aaa'), (2.5, 'Aa1'), (3.5, 'Aa2'), (4.5, 'Aa3'), (5.5, 'A1'), (6.5, 'A2'), (7.5, 'A3'), (8.5, 'Baa1'), (9.5, 'Baa2'), (10.5, 'Baa3'), (11.5, 'Ba1'), (12.5, 'Ba2'), (13.5, 'Ba3'), (14.5, 'B1'), (15.5, 'B2'), (16.5, 'B3'), (17.5, 'Caa1'), (18.5, 'Caa2'), (19.5, 'Caa3')]
    scorecard_indicated_outcome = get_score(numeric_score, scorecard_ranges)
    
    return scorecard_indicated_outcome, numeric_score

In [61]:
def calculate_category_score(metric_values, category_info):
    category_score = 0
    for metric_name, metric_info in category_info['metrics'].items():
        metric_score = calculate_metric_score(metric_values.get(metric_name, {}), metric_info)
        category_score += metric_score * category_info['weight']
    return category_score

def calculate_metric_score(metric_values, metric_info):
    metric_score = 0
    for sub_metric, bounds in metric_info['metrics'].items():
        sub_metric_value = metric_values.get(sub_metric, 0)
        metric_score += score_sub_metric(sub_metric_value, bounds) * metric_info.get('weights', [1]).pop(0)
    return metric_score

def score_sub_metric(value, bounds):
    lower_bound, upper_bound = bounds
    if upper_bound == float('inf') and value >= lower_bound:
        return 1
    elif lower_bound == float('-inf') and value <= upper_bound:
        return 1
    elif lower_bound <= value <= upper_bound:
        return 1
    return 0

def map_score_to_rating(normalized_score):
    if normalized_score >= 90:
        return "Aaa"
    elif normalized_score >= 80:
        return "Aa"
    # Extend this mapping based on your requirements
    else:
        return "C"

def calculate_moody_like_credit_score(metrics, rating_classifier):
    total_score = 0
    total_weight = sum(info['weight'] for info in rating_classifier.values())

    for category, info in rating_classifier.items():
        category_score = calculate_category_score(metrics.get(category, {}), info)
        total_score += category_score

    normalized_score = (total_score / total_weight) * 100
    credit_rating = map_score_to_rating(normalized_score)
    return normalized_score, credit_rating


In [68]:
from typing import Tuple

def get_credit_rating_and_score(
    revenue: float,
    ebita_margin: float,
    debt_to_ebitda: float,
    ebita_to_interest: float,
    rcf_to_net_debt: float
) -> Tuple[str, float]:
    """
    Calculate the credit rating and credit score based on the provided financial ratios.

    Args:
        revenue (float): Total revenue in USD billions.
        ebita_margin (float): EBITA margin as a percentage.
        debt_to_ebitda (float): Debt to EBITDA ratio.
        ebita_to_interest (float): EBITA to Interest Expense ratio.
        rcf_to_net_debt (float): Retained Cash Flow to Net Debt ratio.

    Returns:
        Tuple[str, float]: A tuple containing the credit rating and the credit score.
    """
    # Define the rating thresholds for each factor
    revenue_thresholds = [(60.0, float("inf")), (30.0, 60.0), (10.0, 30.0), (5.0, 10.0),
                           (1.5, 5.0), (0.5, 1.5), (0.2, 0.5), (0.0, 0.2)]
    
    ebita_margin_thresholds = [(0.5, float("inf")), (0.35, 0.5), (0.25, 0.35), (0.2, 0.25),
                                (0.15, 0.2), (0.1, 0.15), (0.05, 0.1), (0.0, 0.05)]
    
    debt_to_ebitda_thresholds = [(0.0, 0.5), (0.5, 1.0), (1.0, 2.0), (2.0, 3.0),
                                  (3.0, 4.5), (4.5, 6.5), (6.5, 9.0), (9.0, float("inf"))]
    ebita_to_interest_thresholds = [(25.0, float("inf")), (15.0, 25.0), (10.0, 15.0), (6.0, 10.0),
                                    (3.0, 6.0), (1.0, 3.0), (0.0, 1.0), (float("-inf"), 0.0)]
    rcf_to_net_debt_thresholds = [(0.8, float("inf")), (0.6, 0.8), (0.4, 0.6), (0.25, 0.4),
                                   (0.15, 0.25), (0.075, 0.15), (0.025, 0.075), (0.0, 0.025)]

    # Map the factor values to their corresponding rating scores
    factor_scores = [
        _map_to_rating_score(revenue, revenue_thresholds),
        _map_to_rating_score(ebita_margin, ebita_margin_thresholds),
        _map_to_rating_score(debt_to_ebitda, debt_to_ebitda_thresholds),
        _map_to_rating_score(ebita_to_interest, ebita_to_interest_thresholds),
        _map_to_rating_score(rcf_to_net_debt, rcf_to_net_debt_thresholds)
    ]

    # Calculate the aggregate numeric score
    aggregate_score = sum(
        weight * score for weight, score in zip(
            [0.2, 0.1, 0.15, 0.15, 0.1], factor_scores
        )
    )

    # Map the aggregate score to the corresponding credit rating
    rating_thresholds = [
        (19.5, "Ca", 20.0), (18.5, "Caa3", 19.0), (17.5, "Caa2", 18.0), (16.5, "Caa1", 17.0),
        (15.5, "B3", 16.0), (14.5, "B2", 15.0), (13.5, "B1", 14.0), (12.5, "Ba3", 13.0),
        (11.5, "Ba2", 12.0), (10.5, "Ba1", 11.0), (9.5, "Baa3", 10.0), (8.5, "Baa2", 9.0),
        (7.5, "Baa1", 8.0), (6.5, "A3", 7.0), (5.5, "A2", 6.0), (4.5, "A1", 5.0),
        (3.5, "Aa3", 4.0), (2.5, "Aa2", 3.0), (1.5, "Aa1", 2.0), (0.0, "Aaa", 1.0)
    ]

    for threshold, rating, score in rating_thresholds:
        if aggregate_score >= threshold:
            return rating, score

def _map_to_rating_score(value: float, thresholds: Tuple[Tuple[float, float], ...]) -> int:
    """
    Map a value to its corresponding rating score based on the provided thresholds.

    Args:
        value (float): The value to be mapped.
        thresholds (Tuple[Tuple[float, float], ...]): A tuple of tuples representing the rating thresholds.

    Returns:
        int: The corresponding rating score.
    """
    for score, (lower, upper) in enumerate(thresholds):
        if lower <= value < upper:
            return score
    return len(thresholds)

In [82]:
credit_rating, credit_score = get_credit_rating_and_score(
    revenue=0.0,
    ebita_margin=0.1,
    debt_to_ebitda=100.5,
    ebita_to_interest=1.0,
    rcf_to_net_debt=0.2
)
print(f"Credit Rating: {credit_rating}")  # Output: Credit Rating: A2
print(f"Credit Score: {credit_score}")    # Output: Credit Score: 6.0

Credit Rating: Aa3
Credit Score: 4.0


In [66]:
from typing import Tuple
from dataclasses import dataclass

@dataclass
class FinancialRatios:
    """
    A dataclass representing the financial ratios required for credit rating calculation.
    """
    revenue: float
    ebita_margin: float
    debt_to_ebitda: float
    ebita_to_interest: float
    rcf_to_net_debt: float

@dataclass
class RatingThreshold:
    """
    A dataclass representing a rating threshold and its corresponding credit score.
    """
    threshold: float
    rating: str
    credit_score: float

def get_credit_rating_and_score(financial_ratios: FinancialRatios) -> Tuple[str, float]:
    """
    Calculate the credit rating and credit score based on the provided financial ratios.

    Args:
        financial_ratios (FinancialRatios): A FinancialRatios object containing the financial ratios.

    Returns:
        Tuple[str, float]: A tuple containing the credit rating and the credit score.
    """
    # Define the rating thresholds for each factor
    revenue_thresholds = [(60.0, float("inf")), (30.0, 60.0), (10.0, 30.0), (5.0, 10.0),
                           (1.5, 5.0), (0.5, 1.5), (0.2, 0.5), (0.0, 0.2)]
    ebita_margin_thresholds = [(0.5, float("inf")), (0.35, 0.5), (0.25, 0.35), (0.2, 0.25),
                                (0.15, 0.2), (0.1, 0.15), (0.05, 0.1), (0.0, 0.05)]
    debt_to_ebitda_thresholds = [(0.0, 0.5), (0.5, 1.0), (1.0, 2.0), (2.0, 3.0),
                                  (3.0, 4.5), (4.5, 6.5), (6.5, 9.0), (9.0, float("inf"))]
    ebita_to_interest_thresholds = [(25.0, float("inf")), (15.0, 25.0), (10.0, 15.0), (6.0, 10.0),
                                    (3.0, 6.0), (1.0, 3.0), (0.0, 1.0), (float("-inf"), 0.0)]
    rcf_to_net_debt_thresholds = [(0.8, float("inf")), (0.6, 0.8), (0.4, 0.6), (0.25, 0.4),
                                   (0.15, 0.25), (0.075, 0.15), (0.025, 0.075), (0.0, 0.025)]

    # Map the factor values to their corresponding rating scores
    factor_scores = [
        _map_to_rating_score(financial_ratios.revenue, revenue_thresholds),
        _map_to_rating_score(financial_ratios.ebita_margin, ebita_margin_thresholds),
        _map_to_rating_score(financial_ratios.debt_to_ebitda, debt_to_ebitda_thresholds),
        _map_to_rating_score(financial_ratios.ebita_to_interest, ebita_to_interest_thresholds),
        _map_to_rating_score(financial_ratios.rcf_to_net_debt, rcf_to_net_debt_thresholds)
    ]

    # Calculate the aggregate numeric score
    aggregate_score = sum(
        weight * score for weight, score in zip(
            [0.2, 0.1, 0.15, 0.15, 0.1], factor_scores
        )
    )

    # Map the aggregate score to the corresponding credit rating and credit score
    rating_thresholds = [
        RatingThreshold(19.5, "Ca", 20.0), RatingThreshold(18.5, "Caa3", 19.0),
        RatingThreshold(17.5, "Caa2", 18.0), RatingThreshold(16.5, "Caa1", 17.0),
        RatingThreshold(15.5, "B3", 16.0), RatingThreshold(14.5, "B2", 15.0),
        RatingThreshold(13.5, "B1", 14.0), RatingThreshold(12.5, "Ba3", 13.0),
        RatingThreshold(11.5, "Ba2", 12.0), RatingThreshold(10.5, "Ba1", 11.0),
        RatingThreshold(9.5, "Baa3", 10.0), RatingThreshold(8.5, "Baa2", 9.0),
        RatingThreshold(7.5, "Baa1", 8.0), RatingThreshold(6.5, "A3", 7.0),
        RatingThreshold(5.5, "A2", 6.0), RatingThreshold(4.5, "A1", 5.0),
        RatingThreshold(3.5, "Aa3", 4.0), RatingThreshold(2.5, "Aa2", 3.0),
        RatingThreshold(1.5, "Aa1", 2.0), RatingThreshold(0.0, "Aaa", 1.0)
    ]

    for threshold in rating_thresholds:
        if aggregate_score >= threshold.threshold:
            return threshold.rating, threshold.credit_score

def _map_to_rating_score(value: float, thresholds: Tuple[Tuple[float, float], ...]) -> int:
    """
    Map a value to its corresponding rating score based on the provided thresholds.

    Args:
        value (float): The value to be mapped.
        thresholds (Tuple[Tuple[float, float], ...]): A tuple of tuples representing the rating thresholds.

    Returns:
        int: The corresponding rating score.
    """
    for score, (lower, upper) in enumerate(thresholds):
        if lower <= value < upper:
            return score
    return len(thresholds)

# Example usage
financial_ratios = FinancialRatios(
    revenue=15.0,
    ebita_margin=0.3,
    debt_to_ebitda=1.5,
    ebita_to_interest=12.0,
    rcf_to_net_debt=0.5
)

credit_rating, credit_score = get_credit_rating_and_score(financial_ratios)
print(f"Credit Rating: {credit_rating}")  # Output: Credit Rating: A2
print(f"Credit Score: {credit_score}")    # Output: Credit Score: 6.0

Credit Rating: Aaa
Credit Score: 1.0


In [58]:
scorecard_outcome, numeric_score = scorecard(
    revenue=15.0,
    demand_characteristics='Mostly steady demand, with moderate exposure to economic cycles',
    competitive_profile='Several business segments with broad service offerings in many segments',
    ebita_margin=30.0,
    debt_ebitda=1.5,
    ebita_interest_expense=12.0,
    rcf_net_debt=50.0,
    financial_policy='Expected to have predictable financial policies that preserve creditor interests'
)

print(f"Scorecard-indicated outcome: {scorecard_outcome}")
print(f"Numeric score: {numeric_score}")

ValueError: could not convert string to float: '50%'